##Read in KDD99 Data Set




In [0]:
import pandas as pd
from tensorflow.keras.utils import get_file

try:
    path = get_file('kddcup.data.gz', origin='http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz')
except:
    print('Error downloading')
    raise
    
print(path) 

# This file is a CSV, just no CSV extension or headers
# Download from: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
df = pd.read_csv(path, header=None)

print("Read {} rows.".format(len(df)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset
df.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)

# The CSV file has no column heads, so add them
df.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome'
]

# display 5 rows
df[0:5]

18120704/18115902 [==============================] - 2s 0us/step
/root/.keras/datasets/kddcup.data.gz
Read 4898431 rows.


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
0,0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [0]:
df.shape

(4898431, 42)

##Analyzing a Dataset

In [0]:
ENCODING = 'utf-8'

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

In [0]:
# Analyze KDD-99

import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

analyze(df)


4898431 rows
** duration:9883 (0%)
** protocol_type:[icmp:57.85%,tcp:38.19%,udp:3.97%]
** service:[ecr_i:57.4%,private:22.47%,http:12.72%,smtp:1.97%,other:1.48%,domain_u:1.18%,ftp_data:0.83%,eco_i:0.33%,finger:0.14%,urp_i:0.11%,ftp:0.11%,telnet:0.09%,ntp_u:0.08%,auth:0.07%,pop_3:0.04%,time:0.03%,domain:0.02%,Z39_50:0.02%,gopher:0.02%,mtp:0.02%,ssh:0.02%,whois:0.02%,remote_job:0.02%,rje:0.02%,imap4:0.02%,link:0.02%,ctf:0.02%,name:0.02%,supdup:0.02%,discard:0.02%,nntp:0.02%,echo:0.02%,uucp_path:0.02%,sunrpc:0.02%,systat:0.02%,netstat:0.02%,daytime:0.02%,pop_2:0.02%,netbios_ssn:0.02%,netbios_ns:0.02%,vmnet:0.02%,iso_tsap:0.02%,sql_net:0.02%,netbios_dgm:0.02%,csnet_ns:0.02%,shell:0.02%,klogin:0.02%,hostnames:0.02%,bgp:0.02%,printer:0.02%,exec:0.02%,login:0.02%,http_443:0.02%,efs:0.02%,ldap:0.02%,uucp:0.02%,kshell:0.02%,nnsp:0.02%,courier:0.02%,IRC:0.01%,urh_i:0.0%,X11:0.0%,tim_i:0.0%,red_i:0.0%,pm_dump:0.0%,tftp_u:0.0%,harvest:0.0%,aol:0.0%,http_8001:0.0%,http_2784:0.0%]
** flag:[SF:76.44

##Encode the feature vector

In [0]:
df.groupby('outcome')['outcome'].count()

outcome
back.                  2203
buffer_overflow.         30
ftp_write.                8
guess_passwd.            53
imap.                    12
ipsweep.              12481
land.                    21
loadmodule.               9
multihop.                 7
neptune.            1072017
nmap.                  2316
normal.              972781
perl.                     3
phf.                      4
pod.                    264
portsweep.            10413
rootkit.                 10
satan.                15892
smurf.              2807886
spy.                      2
teardrop.               979
warezclient.           1020
warezmaster.             20
Name: outcome, dtype: int64

In [0]:
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
    
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

In [0]:
# Now encode the feature vector

encode_numeric_zscore(df, 'duration')
encode_text_dummy(df, 'protocol_type')
encode_text_dummy(df, 'service')
encode_text_dummy(df, 'flag')
encode_numeric_zscore(df, 'src_bytes')
encode_numeric_zscore(df, 'dst_bytes')
encode_text_dummy(df, 'land')
encode_numeric_zscore(df, 'wrong_fragment')
encode_numeric_zscore(df, 'urgent')
encode_numeric_zscore(df, 'hot')
encode_numeric_zscore(df, 'num_failed_logins')
encode_text_dummy(df, 'logged_in')
encode_numeric_zscore(df, 'num_compromised')
encode_numeric_zscore(df, 'root_shell')
encode_numeric_zscore(df, 'su_attempted')
encode_numeric_zscore(df, 'num_root')
encode_numeric_zscore(df, 'num_file_creations')
encode_numeric_zscore(df, 'num_shells')
encode_numeric_zscore(df, 'num_access_files')
encode_numeric_zscore(df, 'num_outbound_cmds')
encode_text_dummy(df, 'is_host_login')
encode_text_dummy(df, 'is_guest_login')
encode_numeric_zscore(df, 'count')
encode_numeric_zscore(df, 'srv_count')
encode_numeric_zscore(df, 'serror_rate')
encode_numeric_zscore(df, 'srv_serror_rate')
encode_numeric_zscore(df, 'rerror_rate')
encode_numeric_zscore(df, 'srv_rerror_rate')
encode_numeric_zscore(df, 'same_srv_rate')
encode_numeric_zscore(df, 'diff_srv_rate')
encode_numeric_zscore(df, 'srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_count')
encode_numeric_zscore(df, 'dst_host_srv_count')
encode_numeric_zscore(df, 'dst_host_same_srv_rate')
encode_numeric_zscore(df, 'dst_host_diff_srv_rate')
encode_numeric_zscore(df, 'dst_host_same_src_port_rate')
encode_numeric_zscore(df, 'dst_host_srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_serror_rate')
encode_numeric_zscore(df, 'dst_host_srv_serror_rate')
encode_numeric_zscore(df, 'dst_host_rerror_rate')
encode_numeric_zscore(df, 'dst_host_srv_rerror_rate')

# display 5 rows

df.dropna(inplace=True,axis=1)
df[0:5]

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome,protocol_type-icmp,protocol_type-tcp,protocol_type-udp,service-IRC,service-X11,service-Z39_50,...,service-private,service-red_i,service-remote_job,service-rje,service-shell,service-smtp,service-sql_net,service-ssh,service-sunrpc,service-supdup,service-systat,service-telnet,service-tftp_u,service-tim_i,service-time,service-urh_i,service-urp_i,service-uucp,service-uucp_path,service-vmnet,service-whois,flag-OTH,flag-REJ,flag-RSTO,flag-RSTOS0,flag-RSTR,flag-S0,flag-S1,flag-S2,flag-S3,flag-SF,flag-SH,land-0,land-1,logged_in-0,logged_in-1,is_host_login-0,is_host_login-1,is_guest_login-0,is_guest_login-1
0,-0.066833,-0.001720,0.068188,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.575415,-1.196243,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.639139,-1.786510,-1.833023,-0.282939,-1.257937,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0
1,-0.066833,-0.001777,0.005325,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.570698,-1.192178,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.623519,-1.777069,0.598967,-0.282939,0.821119,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0
2,-0.066833,-0.001698,0.000208,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.575415,-1.196243,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.607899,-1.767627,0.598967,-0.282939,-0.218409,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0
3,-0.066833,-0.001701,0.001455,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.570698,-1.192178,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.592279,-1.758185,0.598967,-0.282939,-0.571848,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0
4,-0.066833,-0.001695,-0.000942,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.565981,-1.188113,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.576659,-1.748743,0.598967,-0.282939,-0.738173,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0


In [0]:
# This is the numeric feature vector, as it goes to the neural net
# Convert to numpy - Classification
x_columns = df.columns.drop('outcome')
x = df[x_columns].values
dummies = pd.get_dummies(df['outcome']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

##Train the Neural Network

In [0]:

import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

# Create a test/train split.  25% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
114807/114807 - 185s - loss: 0.0252 - val_loss: 0.0312
Epoch 2/1000
114807/114807 - 186s - loss: 0.0132 - val_loss: 0.0373
Epoch 3/1000
114807/114807 - 188s - loss: 0.0116 - val_loss: 0.0136
Epoch 4/1000
114807/114807 - 184s - loss: 0.0132 - val_loss: 0.0311
Epoch 5/1000
114807/114807 - 187s - loss: 0.0123 - val_loss: 0.0453
Epoch 6/1000
114807/114807 - 185s - loss: 0.0141 - val_loss: 0.0080
Epoch 7/1000
114807/114807 - 184s - loss: 0.0092 - val_loss: 0.0157
Epoch 8/1000
114807/114807 - 184s - loss: 0.0115 - val_loss: 0.0282
Epoch 9/1000
114807/114807 - 183s - loss: 0.0090 - val_loss: 0.0080
Epoch 10/1000
114807/114807 - 185s - loss: 0.0128 - val_loss: 0.0419
Epoch 11/1000
114807/114807 - 183s - loss: 0.0106 - val_loss: 0.0468
Epoch 00011: early stopping


In [0]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

Validation score: 0.9985897528025295


In [0]:
df.shape

(4898431, 126)

In [0]:
from datetime import datetime
import numpy as np
date_range =  pd.date_range(start='01/01/1999', periods=4898431, freq='S')

In [0]:
len(date_range)

4898431

In [0]:
df[0:5]

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome,protocol_type-icmp,protocol_type-tcp,protocol_type-udp,service-IRC,service-X11,service-Z39_50,...,service-private,service-red_i,service-remote_job,service-rje,service-shell,service-smtp,service-sql_net,service-ssh,service-sunrpc,service-supdup,service-systat,service-telnet,service-tftp_u,service-tim_i,service-time,service-urh_i,service-urp_i,service-uucp,service-uucp_path,service-vmnet,service-whois,flag-OTH,flag-REJ,flag-RSTO,flag-RSTOS0,flag-RSTR,flag-S0,flag-S1,flag-S2,flag-S3,flag-SF,flag-SH,land-0,land-1,logged_in-0,logged_in-1,is_host_login-0,is_host_login-1,is_guest_login-0,is_guest_login-1
0,-0.066833,-0.001720,0.068188,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.575415,-1.196243,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.639139,-1.786510,-1.833023,-0.282939,-1.257937,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0
1,-0.066833,-0.001777,0.005325,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.570698,-1.192178,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.623519,-1.777069,0.598967,-0.282939,0.821119,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0
2,-0.066833,-0.001698,0.000208,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.575415,-1.196243,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.607899,-1.767627,0.598967,-0.282939,-0.218409,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0
3,-0.066833,-0.001701,0.001455,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.570698,-1.192178,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.592279,-1.758185,0.598967,-0.282939,-0.571848,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0
4,-0.066833,-0.001695,-0.000942,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.565981,-1.188113,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.576659,-1.748743,0.598967,-0.282939,-0.738173,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0


In [0]:
df['timestamp']=date_range

In [0]:
date_range.dtype

dtype('<M8[ns]')

In [0]:
date_range

DatetimeIndex(['1999-01-01 00:00:00', '1999-01-01 00:00:01',
               '1999-01-01 00:00:02', '1999-01-01 00:00:03',
               '1999-01-01 00:00:04', '1999-01-01 00:00:05',
               '1999-01-01 00:00:06', '1999-01-01 00:00:07',
               '1999-01-01 00:00:08', '1999-01-01 00:00:09',
               ...
               '1999-02-26 16:40:21', '1999-02-26 16:40:22',
               '1999-02-26 16:40:23', '1999-02-26 16:40:24',
               '1999-02-26 16:40:25', '1999-02-26 16:40:26',
               '1999-02-26 16:40:27', '1999-02-26 16:40:28',
               '1999-02-26 16:40:29', '1999-02-26 16:40:30'],
              dtype='datetime64[ns]', length=4898431, freq='S')

In [0]:
df[0:5]

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome,protocol_type-icmp,protocol_type-tcp,protocol_type-udp,service-IRC,service-X11,service-Z39_50,...,service-red_i,service-remote_job,service-rje,service-shell,service-smtp,service-sql_net,service-ssh,service-sunrpc,service-supdup,service-systat,service-telnet,service-tftp_u,service-tim_i,service-time,service-urh_i,service-urp_i,service-uucp,service-uucp_path,service-vmnet,service-whois,flag-OTH,flag-REJ,flag-RSTO,flag-RSTOS0,flag-RSTR,flag-S0,flag-S1,flag-S2,flag-S3,flag-SF,flag-SH,land-0,land-1,logged_in-0,logged_in-1,is_host_login-0,is_host_login-1,is_guest_login-0,is_guest_login-1,timestamp
0,-0.066833,-0.001720,0.068188,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.575415,-1.196243,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.639139,-1.786510,-1.833023,-0.282939,-1.257937,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1999-01-01 00:00:00
1,-0.066833,-0.001777,0.005325,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.570698,-1.192178,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.623519,-1.777069,0.598967,-0.282939,0.821119,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1999-01-01 00:00:01
2,-0.066833,-0.001698,0.000208,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.575415,-1.196243,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.607899,-1.767627,0.598967,-0.282939,-0.218409,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1999-01-01 00:00:02
3,-0.066833,-0.001701,0.001455,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.570698,-1.192178,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.592279,-1.758185,0.598967,-0.282939,-0.571848,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1999-01-01 00:00:03
4,-0.066833,-0.001695,-0.000942,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.565981,-1.188113,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.576659,-1.748743,0.598967,-0.282939,-0.738173,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1999-01-01 00:00:04


In [0]:
df.tail()

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome,protocol_type-icmp,protocol_type-tcp,protocol_type-udp,service-IRC,service-X11,service-Z39_50,...,service-red_i,service-remote_job,service-rje,service-shell,service-smtp,service-sql_net,service-ssh,service-sunrpc,service-supdup,service-systat,service-telnet,service-tftp_u,service-tim_i,service-time,service-urh_i,service-urp_i,service-uucp,service-uucp_path,service-vmnet,service-whois,flag-OTH,flag-REJ,flag-RSTO,flag-RSTOS0,flag-RSTR,flag-S0,flag-S1,flag-S2,flag-S3,flag-SF,flag-SH,land-0,land-1,logged_in-0,logged_in-1,is_host_login-0,is_host_login-1,is_guest_login-0,is_guest_login-1,timestamp
4898426,-0.066833,-0.001724,0.001852,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.565981,-1.070223,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,0.937248,-3.592279,0.621131,0.598967,-0.282939,-0.571848,1.055165,-0.466405,-0.439288,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1999-02-26 16:40:26
4898427,-0.066833,-0.001716,-0.001330,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.561263,-1.066158,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,0.866104,-3.576659,0.621131,0.598967,-0.282939,-0.738173,1.055165,-0.466405,-0.439288,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1999-02-26 16:40:27
4898428,-0.066833,-0.001717,0.003901,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.556546,-1.062093,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,0.866104,-3.561039,0.621131,0.598967,-0.282939,-0.842125,1.055165,-0.466405,-0.439288,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1999-02-26 16:40:28
4898429,-0.066833,-0.001716,0.000218,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.551829,-1.058028,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,0.794960,-3.545420,0.621131,0.598967,-0.282939,-0.904497,1.055165,-0.466405,-0.439288,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1999-02-26 16:40:29
4898430,-0.066833,-0.001716,0.000007,-0.015139,-0.001103,-0.026521,-0.004391,-0.002097,-0.008258,-0.004546,-0.003285,-0.009572,-0.008505,-0.028756,-1.575415,-1.196243,-0.466043,-0.465756,-0.248286,-0.24813,0.539733,-0.256056,-0.201059,-3.529800,0.621131,0.598967,-0.282939,-0.966869,1.055165,-0.466405,-0.439288,-0.250832,-0.249632,normal.,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1999-02-26 16:40:30


In [0]:
df.dtypes

duration                   float64
src_bytes                  float64
dst_bytes                  float64
wrong_fragment             float64
urgent                     float64
                         ...      
is_host_login-0              uint8
is_host_login-1              uint8
is_guest_login-0             uint8
is_guest_login-1             uint8
timestamp           datetime64[ns]
Length: 127, dtype: object

In [0]:
#performance analysis
analyze(df)


4898431 rows
** duration:9883 (0%)
** src_bytes:7195 (0%)
** dst_bytes:21493 (0%)
** wrong_fragment:[-0.01513917188874851:99.97%,69.98944442865334:0.02%,23.31972202829195:0.01%]
** urgent:[-0.001103484505593594:100.0%,138.59742754675938:0.0%,277.1959585780243:0.0%,415.79448960928926:0.0%,692.9915516718191:0.0%,1940.3783309532039:0.0%]
** hot:[-0.026520757249818123:99.83%,4.238070115330694:0.05%,2.105774679040438:0.03%,8.502660987911206:0.02%,12.767251860491717:0.02%,10.63495642420146:0.01%,63.94234233145786:0.01%,40.48709253226504:0.01%,59.677751458877346:0.01%,29.825615350813766:0.01%,38.35479709597479:0.01%,46.88397884113581:0.01%,51.14856971371632:0.01%,6.37036555162095:0.0%,14.899547296781973:0.0%,42.619387968555294:0.0%,19.164138169362484:0.0%,31.957910787104023:0.0%,23.428729041942997:0.0%,36.22250165968453:0.0%,21.29643360565274:0.0%,34.090206223394276:0.0%,27.69331991452351:0.0%,44.75168340484555:0.0%,25.561024478233254:0.0%,17.031842733072228:0.0%,93.79447843952144:0.0%,53.28